**Приведение датасета к виду YOLO**

*images_dir* - папка с изображениями

*labels_dir* - папка с метками (в формате yolo) (будет создана автоматически из csv с указанным путем)

*empty_images_dir* - папка с пустыми изображениями

*train_images_dir* и train_labels_dir - места, где будут созданы папки с тренировочной датой

*val_images_dir* и val_labels_dir - места, где будут созданы папки с валидационной датой

In [ ]:
from PIL import Image
import pandas as pd
import os
import shutil
import random

# Загрузим CSV файл с аннотациями
annotations = pd.read_csv('СВОЙ ПУТЬ')

# Пути к изображениям и папке для меток
# СЮДА ВПИСАТЬ СВОИ ПУТИ
images_dir = 'СВОЙ ПУТЬ'
labels_dir = 'СВОЙ ПУТЬ'
empty_images_dir = 'СВОЙ ПУТЬ'  # папка с пустыми изображениями


# Пути для разделенных данных (train и val)
# СЮДА ВПИСАТЬ СВОИ ПУТИ
train_images_dir = 'СВОЙ ПУТЬ'
train_labels_dir = 'СВОЙ ПУТЬ'
val_images_dir = 'СВОЙ ПУТЬ'
val_labels_dir = 'СВОЙ ПУТЬ'

# Создаем папки для меток, если их еще нет
os.makedirs(labels_dir, exist_ok=True)

# Преобразуем координаты из CSV в формат YOLO и сохраняем в .txt файлы
for _, row in annotations.iterrows():
    image_name = row['Name']
    bbox = list(map(float, row['Bbox'].split(',')))
    obj_class = int(row['Class'])  # Получаем класс объекта из столбца "Class"
    x_center, y_center, width, height = bbox


    # Сохранение аннотаций в YOLO формате с учетом класса
    label_path = os.path.join(labels_dir, f"{os.path.splitext(image_name)[0]}.txt")
    with open(label_path, 'w') as f:
        f.write(f"{obj_class} {x_center} {y_center} {width} {height}\n")

# Обработка пустых изображений
for empty_image in os.listdir(empty_images_dir):
    label_path = os.path.join(labels_dir, f"{os.path.splitext(empty_image)[0]}.txt")
    with open(label_path, 'w') as f:
        f.write('')  # Пустой файл для изображений без объектов


# Создаем папки для train и val, если их нет
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Получаем список всех изображений, включая пустые
all_images = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
all_images += [f for f in os.listdir(empty_images_dir) if f.endswith('.jpg')]  # Добавляем пустые изображения

# Перемешиваем список для случайного распределения
random.shuffle(all_images)

# Устанавливаем соотношение для разделения
train_ratio = 0.8  # 80% на обучение, 20% на валидацию
train_count = int(len(all_images) * train_ratio)

# Разделение данных
train_images = all_images[:train_count]
val_images = all_images[train_count:]

# Обновленная функция для копирования файлов изображений и аннотаций с проверкой обеих папок
def copy_files(images, dest_images_dir, dest_labels_dir):
    for image_name in images:
        # Проверяем наличие изображения в обеих папках и выбираем путь оттуда, где оно найдено
        src_image_path = os.path.join(images_dir, image_name) if os.path.exists(
            os.path.join(images_dir, image_name)) else os.path.join(empty_images_dir, image_name)

        # Открываем и сжимаем изображение до 640x640
        with Image.open(src_image_path) as img:
            img_resized = img.resize((1280, 1280))  # Изменение размера
            dest_image_path = os.path.join(dest_images_dir, image_name)
            img_resized.save(dest_image_path)  # Сохранение сжатого изображения

        # Копируем файл аннотации
        label_name = f"{os.path.splitext(image_name)[0]}.txt"
        src_label_path = os.path.join(labels_dir, label_name)
        dest_label_path = os.path.join(dest_labels_dir, label_name)
        if os.path.exists(src_label_path):  # проверка, чтобы избежать ошибок на пустых файлах
            shutil.copyfile(src_label_path, dest_label_path)



# Копируем файлы в train и val директории
copy_files(train_images, train_images_dir, train_labels_dir)
copy_files(val_images, val_images_dir, val_labels_dir)


**Аугментация фото (увеличивает в 7 раз)**

*path_to_original_dataset* и *path_to_finish_dataset* - пути к датасету в формате YOLO и к месту куда будет сохранен расширенный датасет

*annotations_path* - папка с метками (в формате yolo) (будет создана автоматически из csv с указанным путем)

*empty_images_dir* - папка с пустыми изображениями

*images_dir* - папка с фото

*labels_dir* - папка с метками (в формате yolo)

In [ ]:
import pandas as pd
import os
import shutil
import random
from PIL import Image
import albumentations as A
import cv2
import numpy as np

# Пути к исходному и конечному датасету
path_to_original_dataset = 'СВОЙ ПУТЬ'
path_to_finish_dataset = 'СВОЙ ПУТЬ'

# Пути к данным
annotations_path = os.path.join(path_to_original_dataset, 'annotation.csv')
images_dir = os.path.join(path_to_original_dataset, 'images')
empty_images_dir = os.path.join(path_to_original_dataset, 'images_empty')
labels_dir = os.path.join(path_to_original_dataset, 'labels')

# Создаем папку для аннотаций, если её ещё нет
os.makedirs(labels_dir, exist_ok=True)

# Преобразуем координаты из CSV в формат YOLO и сохраняем в .txt файлы
annotations = pd.read_csv(annotations_path)
for _, row in annotations.iterrows():
    image_name = row['Name']
    bbox = list(map(float, row['Bbox'].split(',')))
    obj_class = int(row['Class'])
    x_center, y_center, width, height = bbox

    # Сохранение аннотаций в YOLO формате
    label_path = os.path.join(labels_dir, f"{os.path.splitext(image_name)[0]}.txt")
    with open(label_path, 'w') as f:
        f.write(f"{obj_class} {x_center} {y_center} {width} {height}\n")

# Создаем папки для расширенного датасета
train_images_dir = os.path.join(path_to_finish_dataset, 'images/train')
train_labels_dir = os.path.join(path_to_finish_dataset, 'labels/train')
val_images_dir = os.path.join(path_to_finish_dataset, 'images/val')
val_labels_dir = os.path.join(path_to_finish_dataset, 'labels/val')

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Определяем слабые и сильные аугментации
weak_augmentations = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
    A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=5, val_shift_limit=5, p=0.5),
    A.Blur(blur_limit=(1, 3), p=0.2),
    A.VerticalFlip(p=0.5),
    A.Resize(1280, 1280)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

strong_augmentations = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.5, p=0.7),
    A.HueSaturationValue(hue_shift_limit=50, sat_shift_limit=100, val_shift_limit=100, p=0.7),
    A.GaussNoise(var_limit=(20.0, 50.0), p=0.3),
    A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.6, p=0.3),
    A.RandomRain(blur_value=3, p=0.2),
    A.VerticalFlip(p=0.5),
    A.Resize(1280, 1280)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

empty_augmentations = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.5),
    A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
    A.GaussNoise(var_limit=(10.0, 30.0), p=0.3),
    A.Blur(blur_limit=3, p=0.3),
    A.HorizontalFlip(p=0.5),
    A.Resize(1280, 1280)
])

# Создаем список изображений с указанием их расположения
all_images = [(f, images_dir) for f in os.listdir(images_dir) if f.endswith('.jpg')]
all_images.extend([(f, empty_images_dir) for f in os.listdir(empty_images_dir) if f.endswith('.jpg')])

# Перемешиваем и делим на train и val
random.shuffle(all_images)
train_ratio = 0.7
train_count = int(len(all_images) * train_ratio)

train_images = all_images[:train_count]
val_images = all_images[train_count:]


# Функция для копирования оригинальных изображений и меток
def copy_files(images, dest_images_dir, dest_labels_dir):
    for image_name, src_dir in images:
        src_image_path = os.path.join(src_dir, image_name)
        dest_image_path = os.path.join(dest_images_dir, image_name)
        shutil.copyfile(src_image_path, dest_image_path)
        
        # Если изображение из основной папки, копируем метку
        if src_dir == images_dir:
            label_name = f"{os.path.splitext(image_name)[0]}.txt"
            src_label_path = os.path.join(labels_dir, label_name)
            dest_label_path = os.path.join(dest_labels_dir, label_name)
            if os.path.exists(src_label_path):
                shutil.copyfile(src_label_path, dest_label_path)
        else:
            # Создаем пустой файл метки для пустых изображений
            empty_label_path = os.path.join(dest_labels_dir, f"{os.path.splitext(image_name)[0]}.txt")
            with open(empty_label_path, 'w') as f:
                f.write('')

# Копируем данные в train и val
copy_files(train_images, train_images_dir, train_labels_dir)
copy_files(val_images, val_images_dir, val_labels_dir)

print("Оригинальные данные успешно разделены на обучающую и тестовую выборки.")

# Аугментация изображений в обучающей выборке
for idx, (image_name, src_dir) in enumerate(train_images):
    image_path = os.path.join(src_dir, image_name)
    if not os.path.exists(image_path):
        continue

    if src_dir == empty_images_dir:
        # Аугментация для пустых изображений
        with Image.open(image_path) as img:
            img = img.resize((1280, 1280))
            for aug_num in range(6):
                aug_image = empty_augmentations(image=np.array(img))['image']
                aug_image_name = f"{os.path.splitext(image_name)[0]}_aug_empty_{aug_num}.jpg"
                aug_image_path = os.path.join(train_images_dir, aug_image_name)
                aug_image_pil = Image.fromarray(aug_image)
                aug_image_pil.save(aug_image_path)
                
                label_path = os.path.join(train_labels_dir, f"{os.path.splitext(aug_image_name)[0]}.txt")
                with open(label_path, 'w') as f:
                    f.write('')  # Пустой файл для изображений без объектов
    else:
        # Аугментация для изображений с объектами
        annotation_row = annotations[annotations['Name'] == image_name]
        if annotation_row.empty:
            continue
        
        bbox = list(map(float, annotation_row.iloc[0]['Bbox'].split(',')))
        obj_class = int(annotation_row.iloc[0]['Class'])
        
        image = cv2.imread(image_path)
        if image is None:
            continue
        
        for i in range(7):  # Слабая аугментация
            augmented = weak_augmentations(image=image, bboxes=[bbox], class_labels=[obj_class])
            if 'bboxes' in augmented and augmented['bboxes']:
                aug_bbox = augmented['bboxes'][0]
                aug_image_name = f"{os.path.splitext(image_name)[0]}_aug_weak_{i}.jpg"
                aug_image_path = os.path.join(train_images_dir, aug_image_name)
                cv2.imwrite(aug_image_path, augmented['image'])

                label_path = os.path.join(train_labels_dir, f"{os.path.splitext(aug_image_name)[0]}.txt")
                with open(label_path, 'w') as f:
                    x_center, y_center, width, height = aug_bbox
                    f.write(f"{obj_class} {x_center} {y_center} {width} {height}\n")

        for i in range(7):  # Сильная аугментация
            augmented = strong_augmentations(image=image, bboxes=[bbox], class_labels=[obj_class])
            if 'bboxes' in augmented and augmented['bboxes']:
                aug_bbox = augmented['bboxes'][0]
                aug_image_name = f"{os.path.splitext(image_name)[0]}_aug_strong_{i}.jpg"
                aug_image_path = os.path.join(train_images_dir, aug_image_name)
                cv2.imwrite(aug_image_path, augmented['image'])


                label_path = os.path.join(train_labels_dir, f"{os.path.splitext(aug_image_name)[0]}.txt")
                with open(label_path, 'w') as f:
                    x_center, y_center, width, height = aug_bbox
                    f.write(f"{obj_class} {x_center} {y_center} {width} {height}\n")

print("Датасет успешно увеличен до четырёхкратного размера с нужными аугментациями для всех изображений.")

**Дообучение модели**

Данный код уже имеет оптимальные параметры - достаточно лишь вписать путь к нашей модели и путь к датасету

In [ ]:
os.chdir("Путь/к/папке/с/yolov5")

!python train.py --weights Путь/к/модели/best.pt --epochs 50 --batch-size 25 --device 0 --data Путь/к/датасету/dataset.yaml --imgsz 1280 --freeze 15